# Problème des reines sur échiquier

## Genetic algorithm

In [12]:
import random as rd
import numpy as np
import matplotlib.pyplot as plt 

## 1. Solutions

- **Solutions** : Toutes les cases sont utilisées
- **Solution** optimales : Il y a 0 conflit

## 2. Implémentation

In [ ]:
"""
We init the board as a 2D array of 0
0 means no queen
1 means a queen is present
2 means the cell is attacked by a queen
"""
def board(n):
    return [[0 for _ in range(n)] for _ in range(n)]

def list_to_board(index_list):
    n = len(index_list)
    board = [[0 for _ in range(n)] for _ in range(n)]
    for i, j in enumerate(index_list):
        board[i][j] = 1
    return board

## 3. Fonction objectif

In [4]:
"""
Transform the board to a list of queen
Index = nb line
Value = nb column
"""
def index_queen(board):
    index = []
    for i in range(len(board)):
        for j in range(len(board)):
            if board[i][j] == 1:
                index.append((i, j))
                
    # 0. nb of queens should size board
    if len(index) != len(board):
        return False
    
    return index

"""
Objective function: we want the optimal solution
For that we need to check the diagonal, horizontal and vertical lines
"""
def objective_function(index_list):
    # 1. No queen in corners
    if index_list[0] == 0:
        return False
    if index_list[-1] == len(index_list) - 1:
        return False
    if index_list[0] == len(index_list) - 1:
        return False
    if index_list[-1] == 0:
        return False
         
    # 2. A queen is present in the same column
    for i in range(len(index_list)):
        for j in range(i+1, len(index_list)):
            if index_list[i] == index_list[j]:
                return False
    
    # 3. A queen is present in the same row
    for i in range(len(index_list)):
        for j in range(i+1, len(index_list)):
            if abs(index_list[i] - index_list[j]) == abs(i - j):
                return False

## 4. Croisement

In [8]:
"""
Crossing over
we take two parents and we create a child
taking the first half of the first parent
and the second half of the second parent
"""
def crossing_over(index_list1, index_list2):
    n = len(index_list1)
    half = n // 2

    # Child 1: first half from parent 1, rest from parent 2 in order
    child1 = list(index_list1[:half])
    used1 = set(child1)
    # We add the remaining values from the other parent in order
    for val in index_list2:
        if val not in used1:
            child1.append(val)
            used1.add(val)

    # Child 2: first half from parent 2, rest from parent 1 in order
    child2 = list(index_list2[:half])
    used2 = set(child2)
    # We add the remaining values from the other parent in order
    for val in index_list1:
        if val not in used2:
            child2.append(val)
            used2.add(val)

    return child1, child2
    

## 5. Mutation

In [ ]:
"""
Permutation of two random values in the list
"""
def mutation(index_list):
    n = len(index_list)
    i, j = rd.sample(range(n), 2)
    index_list[i], index_list[j] = index_list[j], index_list[i]

## 6. Genetic algorithm

## Run

In [10]:
board = board(8)
print(board)

[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
